In [102]:
import pandas as pd
import numpy as np
import os
import re
import time

import requests
from bs4 import BeautifulSoup

from scrap_utils import Quote, convert_manual_author, extrat_quote, tokenize, get_babelio_citation, scrap_recent_babelio
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

def get_poem(soup):
    for ii, card in enumerate(soup.find_all('article', class_='entry-item')): # 'entry-description')):
        quote = card.p.text
        title = card.header.h1.text
        author = card.header.span.text[4:]
    return title, quote, author

url =  [ # "https://www.poemes.co/paul-eluard.html",
        "https://www.poemes.co/jacques-prevert.html",
        "https://www.poemes.co/paul-verlaine.html",
        "https://www.poemes.co/victor-hugo.html",
        "https://www.poemes.co/charles-baudelaire.html",
        "https://www.poemes.co/alphonse-de-lamartine.html",
        "https://www.poemes.co/rainer-maria-rilke.html",
        "https://www.poemes.co/rene-francois-sully-prudhomme.html",
        "https://www.poemes.co/robert-desnos.html",
        "https://www.poemes.co/charles-leconte-de-lisle.html",
        "https://www.poemes.co/etienne-de-la-boetie.html",
        "https://www.poemes.co/guillaume-apollinaire.html",
        "https://www.poemes.co/rene-char.html",]

In [ ]:
for url_author in url:
    print(url_author)
    list_lien_titre = []
    old_list_lien_titre = []
    for n_page in range(100):
        url = f"{url_author}?page={n_page}"
        response = requests.get(url, headers=headers)
        html_content = response.content # .decode('utf-8')
        soup = BeautifulSoup(html_content, 'html.parser')
        print(n_page, end = ".")
        new_list_lien_titre = []
        for ii, card in enumerate(soup.find_all('div', class_="widget kopa-article-list-widget categories-list-1")):
            for lien in card.find_all('ul'):
                new_list_lien_titre.append(lien.a.attrs['href'])

        if ".".join(new_list_lien_titre) == ".".join(old_list_lien_titre):
            break
        else:
            old_list_lien_titre = new_list_lien_titre
            list_lien_titre += new_list_lien_titre
            time.sleep(1)
            
    pd.DataFrame(list_lien_titre).to_csv(f"data_v2/scrap_poemesco/{url_author.split('/')[-1][:-4]}.csv", index = False)
    print("-")
    

https://www.poemes.co/jacques-prevert.html
0.1.

In [ ]:
l_author_ = os.listdir("data_v2/scrap_poemesco")

for author_ in l_author_:
    l_quote = []
    print(author_)
    df = pd.read_csv(f"data_v2/scrap_poemesco/{author_}")
    list_titre = df[df.columns[0]]
    for ii, url_titre in enumerate(list_titre):
        print(ii, end = '.')
        url = "https://www.poemes.co/" + url_titre
        response = requests.get(url, headers=headers)
        html_content = response.content # .decode('utf-8')
        soup = BeautifulSoup(html_content, 'html.parser')
        try:
            title, quote, author = get_poem(soup)
            q = Quote(quote, author = author, title = title, source = "www.poemes.co", url = url)
            l_quote.append(q)
            time.sleep(10)
            pd.DataFrame([quote.__dict__ for quote in l_quote]).to_csv(f"data_v2/scrap_poemesco_poem/{author_}.csv", index = False)
        except Exception as e:
            print('-')
            print('error:', ii, '/', len(list_titre))
            print(url, e)
            print("-")
            
    pd.DataFrame([quote.__dict__ for quote in l_quote]).to_csv(f"data_v2/scrap_poemesco_poem/{author_}.csv", index = False)
    print("-")
    print("-")

paul-eluard..csv
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.

UnboundLocalError: local variable 'title' referenced before assignment